# Mean (Boxcar) and Median Filters

Window size: 5x5

In [1]:
import cv2 
import matplotlib.pyplot as plt 
import pandas as pd
import os
import numpy as np
from scipy import ndimage
from astropy.io import fits
from astropy.convolution import convolve as ap_convolve
from astropy.convolution import Box2DKernel

In [2]:
input_dir = '../results/selected_images'
input_files = []
for file in os.listdir(input_dir):
    name, ext = os.path.splitext(file)
    if ext == '.png':
        input_files.append(os.path.join(input_dir, file))
input_files

['../results/selected_images/sentinel-1_image_VH_2018-08-12.png',
 '../results/selected_images/sentinel-1_image_VH_2018-12-10.png',
 '../results/selected_images/sentinel-1_image_VH_2018-10-11.png',
 '../results/selected_images/sentinel-1_image_VH_2019-04-09.png',
 '../results/selected_images/sentinel-1_image_VH_2019-03-04.png',
 '../results/selected_images/sentinel-1_image_VH_2018-09-05.png',
 '../results/selected_images/sentinel-1_image_VH_2019-01-03.png',
 '../results/selected_images/sentinel-1_image_VH_2019-02-08.png',
 '../results/selected_images/sentinel-1_image_VH_2018-11-04.png']

In [3]:
def apply_boxcar(image, kernel_size):
    return ap_convolve(image, Box2DKernel(kernel_size), normalize_kernel=True)

def apply_median(image, kernel_size):
    return ndimage.median_filter(image, size=kernel_size)

def lee_filter(img, size):
    """
    Function from StackOverflow: https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python
    """
    img_mean = ndimage.uniform_filter(img, (size, size))
    img_sqr_mean = ndimage.uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = ndimage.variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [4]:
output_dir = '../results/'
filters = {
    'boxcar': apply_boxcar,
    'median': apply_median, 
}
kernel_size = 5

for filter_name in filters:
    # Output Directory 
    current_output_dir = os.path.join(output_dir, filter_name)
    os.makedirs(current_output_dir, exist_ok=True)

    # # Output Files
    output_files = [] 
    for file in input_files:
        file = os.path.relpath(file, input_dir)
        name, ext = os.path.splitext(file)
        output_files.append(os.path.join(current_output_dir, file))


    # Applying filter
    filter = filters[filter_name]
    input_output_mapping = list(zip(input_files, output_files))
    for input_file, output_file in input_output_mapping:
        print(f'{filter_name}(input_file={input_file}, output_file={output_file})')
        
        if os.path.exists(output_file):
            print(f'>>> Skipping step... This file exists.')
        else:
            image = cv2.imread(input_file, cv2.IMREAD_GRAYSCALE) 
            filtered_image = filter(image, kernel_size)
            cv2.imwrite(output_file, filtered_image)
            # plt.figure()
            # plt.imshow(result, cmap='gray')

boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2018-08-12.png, output_file=../results/boxcar/sentinel-1_image_VH_2018-08-12.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2018-12-10.png, output_file=../results/boxcar/sentinel-1_image_VH_2018-12-10.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2018-10-11.png, output_file=../results/boxcar/sentinel-1_image_VH_2018-10-11.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2019-04-09.png, output_file=../results/boxcar/sentinel-1_image_VH_2019-04-09.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2019-03-04.png, output_file=../results/boxcar/sentinel-1_image_VH_2019-03-04.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2018-09-05.png, output_file=../results/boxcar/sentinel-1_image_VH_2018-09-05.png)
boxcar(input_file=../results/selected_images/sentinel-1_image_VH_2019-01-03.png, output_file=../results/boxcar/sentine